In [7]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
from Agents.DQNAgent import DQNAgent
from Boards.Full_Board import FullBoard

query="""
select *
from `analytics-156605.barath.ludo_table`
order by division, time_stamp
"""

df = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

Downloading:   0%|          | 0/144 [00:00<?, ?rows/s]

In [8]:
user_ids = []
dice = []
pos = []

for i in range (len(df)):
    temp_user_id = df['from_user'][i]
    temps = json.loads(json.loads(df['race'][i])['ludoBoardData'])['upd']
    temp_dice = int(df['dice_score'][i])

    for j, temp in enumerate(temps):
        if j == 0:
            temp_pos = temps[temp]['pos']
        else:
            temp_pos += temps[temp]['pos']
        if temp == temp_user_id:
            temp_user_id = j
        if i == len(df) - 1:
            if j == 0:
                score = [temps[temp]['totScore']]
            else:
                score += [temps[temp]['totScore']]

    user_ids.append(temp_user_id)
    dice.append(temp_dice)
    pos.append(temp_pos)

In [9]:
user_ids = user_ids[1:]
dice = dice[1:]

In [4]:
user_ids = np.array(user_ids)
dice = np.array(dice)
pos = np.array(pos)
score = np.array(score)

pos = pos - pos[0,:]

In [10]:
pos = np.array(pos)

In [14]:
pos[:,12:16]

array([[14, 14, 14, 14],
       [14, 14, 14, 14],
       [14, 14, 14, 14],
       [14, 14, 19, 14],
       [14, 14, 19, 14],
       [14, 14, 19, 14],
       [14, 14, 19, 14],
       [14, 14, 24, 14],
       [14, 14, 24, 14],
       [14, 14, 24, 14],
       [14, 14, 24, 14],
       [14, 14, 24, 19],
       [14, 14, 24, 19],
       [14, 14, 24, 19],
       [14, 14, 24, 19],
       [14, 14, 24, 19],
       [14, 14, 24, 19],
       [14, 14, 24, 25],
       [14, 18, 24, 25],
       [14, 18, 24, 25],
       [14, 18, 24, 25],
       [14, 18, 24, 25],
       [14, 18, 24, 31],
       [14, 18, 24, 32],
       [14, 18, 24, 32],
       [14, 18, 24, 14],
       [14, 18, 24, 14],
       [14, 18, 24, 14],
       [14, 18, 24, 14],
       [14, 14, 24, 14],
       [14, 14, 24, 14],
       [14, 14, 24, 14],
       [14, 14, 24, 14],
       [18, 14, 24, 14],
       [18, 14, 24, 14],
       [18, 14, 24, 14],
       [18, 14, 24, 14],
       [18, 14, 24, 14],
       [22, 14, 24, 14],
       [22, 14, 24, 14],


In [5]:
action = np.argmax((pos[1:] - pos[:-1]), axis=-1) % 4

In [6]:
home = (pos == 56).reshape(-1,4,4).sum(axis=2)
start = np.zeros_like(home)

last = np.expand_dims(start[:,0],-1)
last = np.concatenate((last, np.expand_dims(home[:,0],-1)), -1)

for i in range(1,4):
    last = np.concatenate((last, np.expand_dims(start[:,i],-1)), -1)
    last = np.concatenate((last, np.expand_dims(home[:,i],-1)), -1)

In [7]:
pos = 56 - (-1*pos)
states = np.concatenate((np.expand_dims(dice, -1), pos[:-1,:], last[:-1,:]), -1)

In [8]:
env = FullBoard()
agent0 = DQNAgent(env.state_size(), env.action_size())

agent0.load('./model_output/DQN_4p/0001/weights_0900.hdf5')

2022-09-14 13:13:34.193142: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-14 13:13:34.194298: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-14 13:13:34.351245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-14 13:13:34.352162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: NVIDIA Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-09-14 13:13:34.352264: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
20

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25, 32)            64        
_________________________________________________________________
dense_1 (Dense)              (None, 25, 16)            528       
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1604      
Total params: 2,196
Trainable params: 2,196
Non-trainable params: 0
_________________________________________________________________


In [9]:
q = []

for i in range (len(states)):
    s = states[i,:]
    s = env.convert_state(user_ids[i], s=s.tolist())
    pred = np.squeeze(agent0.model.predict(np.reshape(s, [1,25,1])/62))
    q.append(pred[action[i]])

q = np.array(q)

2022-09-14 13:13:37.650972: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-14 13:13:37.651402: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz
2022-09-14 13:13:37.757707: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [10]:
print([np.mean(q[user_ids==0]), np.mean(q[user_ids==1]), np.mean(q[user_ids==2]), np.mean(q[user_ids==3])])

[0.18343264, 0.2868495, 0.16391188, 0.1662589]


In [11]:
print(score)

[216 110 128  50]


In [1]:
from Boards.Full_Board import FullBoard
env = FullBoard()

s, _, game_over, player_turn = env.reset()

In [2]:
s_ = env.convert_state(player_turn)

In [3]:
s_

[5,
 56,
 56,
 56,
 56,
 43,
 43,
 43,
 43,
 30,
 30,
 30,
 30,
 17,
 17,
 17,
 17,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [4]:
s

[5,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [41]:
state = [1, 5, 55, 15, 20, 25, 30, 35, 40, 45, 55, 50, 60, 65, 70, 75, 80, 0, 0, 0, 0]
players = 4
pieces = 4
pos = 55
p = 0

In [42]:
b = [i for i, e in enumerate(state) if e == pos and i > 0 and i < (players * pieces + 1) 
                            and int((i-1) / pieces) != p]

In [22]:
b = [i for i, e in enumerate(a) if e == 4]

In [23]:
b == []

True

In [36]:
len([])

0

In [43]:
b[0]

10

In [47]:
[i for i, e in enumerate([0, 0, 0, 0]) if e == 4]

[]

In [49]:
1 in [2,2,3]

False

In [1]:
a = [1,2,3]

In [2]:
a[2] += 2

In [3]:
a

[1, 2, 5]